# Реентерабельность

Функция повторного входа может потребоваться более чем одной задачей, не опасаясь повреждения данных.  
Функция повнорного входа может быть прервана в любое время и возоблена позже, без потери данных.  
Реентерабельные функции используют либо локальные переменные, либо защищают данные при использовании глобальных переменных.

Пример функции повторного входа:
``` C++
void strcpy(char *dest, char *src)
{
    while (*dest++ = *src++);
    
    *dest = NULL;
}
```

Здесь копии фргументов `strcpy` помещены в стек задачи, `strcpy` может быть вызвана несколькими заадчиам, не опасаясь, что задачи будут искажать указатели друг друга.

Пример нереентерабельной функции:
``` C++
int Temp;

void swap(int *X, int *Y)
{
    Temp = *X;
    *X = *Y;
    *Y = Temp;
}
```

Функция `swap` меняет содержимое двух аргументов.  
Условия применения:
- Вытесняющее ядро
- Прерывания включены
- Temp объявлена как глобальное целое число

# Нереентерабельная функция

![](./images/not-reenterable-example.png)

Когда `swap` прерывается, `Temp` содержит 1.  
Обработчик прерывания (ISR) создает задачу с более высоким приоритетом, готовой к запуску, поэтому при завершениии прерывания ядро вызывается для переключения на эту задачу. Задача с высоким приоритетом устанавливает `Temp` на 1 и правильно изменяет содержимое своих переменных.  
Задача с высоким приоритетом в конечном итоге отказывается от управления задачей с низким приоритетом, вызывая службу ядра, чтобы задержать себя за один такт.  
Таким образом, задача с более низким приоритетом возобновляется. Обратите внимание, что на данный момент `Temp` по-прежнему установлен в 3. Когда задача с низким приоритетом возобновляет выполнение, она устанавливает `y` в 3, вместо 1.

Если прерывание происходит **до** или **после** `swap`, значения `x` и `y` для обеих задач будут правильными.

# Преобразование кода в реентерабельный

Различные способы сделать код из приведенного примера реентерабельным:
- Объявить `Temp` локальной для `swap`.
- Отключить прерывания перед операцией и включить их позже.
- Использовать семафор.

Не всякий код так просто преобразовать. Ошибка, вызванная нереентерной функцией, может не отображаться в вашем приложении на этапе тестирования, это, скорее всего произойдет, как только продукт будет эксплуатироваться.

Если вы новичек в многозадачности, вам нужно быть осторожным при использовании невозвратных функций.